<a href="https://colab.research.google.com/github/supriyag123/PHD_Pub/blob/main/Streamlittest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 839.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.6 MB/s eta 0:00:00


In [5]:
%%writefile streamlit_app.py
import streamlit as st
import pandas as pd
import time
import numpy as np
import datetime as dt

def create_dummy_df(num_rows=100, num_sensors=5, anomaly_prob=0.05, seed=42):
    np.random.seed(seed)
    timestamps = [dt.datetime.now() + dt.timedelta(seconds=i) for i in range(num_rows)]
    data = {}
    for s in range(1, num_sensors + 1):
        base = np.random.normal(loc=50, scale=5, size=num_rows)
        anomalies = np.random.rand(num_rows) < anomaly_prob
        base[anomalies] += np.random.normal(loc=20, scale=5, size=anomalies.sum())
        data[f"sensor_{s}"] = base
    df = pd.DataFrame(data)
    df.insert(0, "timestamp", timestamps)
    return df

st.set_page_config(layout="wide")
speed = st.sidebar.slider("Playback speed (x)", 0.1, 5.0, 1.0)
window = st.sidebar.number_input("Window size (seconds)", 5, 60, 20)

placeholder = st.empty()
df = create_dummy_df(200, 5)

for i in range(len(df)):
    cur_time = df['timestamp'].iloc[i]
    cutoff = cur_time - pd.Timedelta(seconds=window)
    window_df = df[df['timestamp'] >= cutoff]

    with placeholder.container():
        st.line_chart(window_df.set_index('timestamp'))
        st.metric("Rows streamed", i+1)

    time.sleep(1.0 / speed)


Overwriting streamlit_app.py


In [6]:
from pyngrok import ngrok
ngrok.kill()  # clean old tunnels

# ⬇️ paste your token between the quotes
ngrok.set_auth_token("30zHX6xv0Aqb6STXjMtLNsaa6el_45Xmu5rKv77UofrqjgkFR")

public_url = ngrok.connect(8501)  # tip: region option possible e.g. options={"region":"au"}
print("Streamlit URL:", public_url)

ERROR:pyngrok.process.ngrok:t=2025-08-08T06:23:10+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n"
ERROR:pyngrok.process.ngrok:t=2025-08-08T06:23:10+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok st

PyngrokNgrokError: The ngrok process errored on start: authentication failed: Your account is limited to 1 simultaneous ngrok agent sessions.\nYou can run multiple simultaneous tunnels from a single agent session by defining the tunnels in your agent configuration file and starting them with the command `ngrok start --all`.\nRead more about the agent configuration file: https://ngrok.com/docs/secure-tunnels/ngrok-agent/reference/config\nYou can view your current agent sessions in the dashboard:\nhttps://dashboard.ngrok.com/agents\r\n\r\nERR_NGROK_108\r\n.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# New Section